In [1]:
import torch

Create a tensor and set requires_grad=True to track computation with it

In [3]:
x=torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:

In [5]:
y=x+2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y was created as result of an operation, so it has a grad_fn

In [6]:
print(y.grad_fn)

Do more operation on y

In [7]:
z=y*y*3
out=z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_( ... ) changes an existing Tensor’s requires_grad flag in-place. The input flag defaults to False if not given.

In [8]:
a=torch.rand(2,2)
a=((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)

False
True


Gradients

Let’s backprop now. Because out contains a single scalar, out.backward() is equivalent to out.backward(torch.tensor(1.))

In [9]:
out.backward()

Print gradients d(out)/dx

In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Example of a vector-Jacobian product

In [11]:
x=torch.rand(3,requires_grad=True)

y=x*2
while y.data.norm()<1000:
    y=y*2
print(y)

tensor([600.1627, 741.4570, 821.7700], grad_fn=<MulBackward0>)


Now in this case y is no longer a scalar. torch.autograd could not compute the full Jacobian directly, but if we just want the vector-Jacobian product, simply pass the vector to backward as argument:

In [13]:
v=torch.tensor([0.1,1.0,0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


You can also stop autograd from tracking history on Tensors with .requires_grad=True either by wrapping the code block in with torch.no_grad():

In [14]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


Or by using .detach() to get a new Tensor with the same content but that does not require gradients:

In [15]:
print(x.requires_grad)
y=x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
